<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-06 14:30:12
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.38 C
-------------------
Today PnL: -74.92 K (-0.54%)
Current PnL: -31.25 L (-19.93%)
CY Booked + Current PnL: -16.51 L (-10.53%)
-------------------
Total profit:  1.09 L
Total loss:  -32.34 L
-------------------
Total Booked + Current PnL: 10.47 L (8.06%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.66%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 96.64 L (69.9%)
Deployed:  1.30 C
Current:  1.38 C
CAGR/XIRR %: 3.27%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.49,5.68,12.82,19.23,20385.0,8548.0,159008.0,484.83,24.27,53.0,L-LC,12.42,182.0,0.42,1.15,28.16,XY25,ATH,MINING
50,MASFIN,2.22,-1.87,23.88,21.57,22961.0,-1830.0,96150.0,397.04,-15.54,54.0,H-SC,3.25,168.0,-0.08,0.70,40.31,XR,ATH,FINANCE
77,TTKPRESTIG,1.63,-26.42,36.03,0.09,26717.0,-26624.0,74153.0,770.00,72.04,40.0,M-SC,8.21,253.0,-1.00,0.54,2.06,OX40N,NTT,DURABLES
51,MEDANTA,2.97,-4.76,30.28,24.08,36610.0,-6046.0,120904.0,1486.00,-12.55,53.0,X-SC,11.28,91.0,-0.17,0.87,10.74,XY24,NTT,HEALTHCARE
57,RECLTD,-1.74,0.42,20.23,20.74,41274.0,852.0,204022.0,446.00,33.49,55.0,M-MC,4.01,189.0,0.02,1.48,12.43,XY25,NTT,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,4.22,-20.76,39.81,10.78,75808.0,-49894.0,190424.0,452.00,-52.60,43.0,X-LC,18.74,1.0,-0.66,1.38,4.89,X40,NTT,FMCG
26,FINCABLES,3.11,-2.64,113.55,107.91,163225.0,-3897.0,143747.0,1641.55,-20.96,58.0,M-SC,11.55,220.0,-0.02,1.04,8.15,OX40N,ATH,CABLES
51,MEDANTA,2.97,-4.76,30.28,24.08,36610.0,-6046.0,120904.0,1486.00,-12.55,53.0,X-SC,11.28,91.0,-0.17,0.87,10.74,XY24,NTT,HEALTHCARE
7,ATULAUTO,2.40,-23.77,90.95,45.57,136680.0,-46852.0,150280.0,844.00,3245.95,58.0,M-SC,9.36,245.0,-0.34,1.09,13.73,XY24,NTT,AUTO
83,WHIRLPOOL,2.25,-31.79,166.71,81.92,130536.0,-36497.0,78301.0,2270.00,-60.10,55.0,M-SC,32.92,218.0,-0.28,0.57,9.99,XR,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-5.09,-20.76,90.28,50.79,100626.0,-29193.0,111460.0,831.7,1.66,54.0,H-SC,6.84,171.0,-0.29,0.81,29.78,XR,ATH,IT
55,RAJESHEXPO,-5.00,-66.87,202.01,0.07,91820.0,-91724.0,45453.0,518.0,1565.40,49.0,L-SC,11.42,269.0,-1.00,0.33,13.04,OX40N,NTT,JEWELLERY
72,TATAELXSI,-4.70,-25.75,77.81,32.03,76167.0,-33947.0,97888.0,9161.0,-19.73,39.0,H-SC,6.38,157.0,-0.45,0.71,9.64,OX40N,NTT,IT
63,SFL,-3.55,-42.06,118.54,26.63,180105.0,-110282.0,151936.0,1287.0,15.05,58.0,M-SC,16.07,241.0,-0.61,1.10,16.63,XY24,NTT,MISC
65,SIEMENS,-2.88,-13.89,45.76,25.51,73330.0,-25845.0,160250.0,4671.5,-1.60,59.0,H-LC,3.54,50.0,-0.35,1.16,18.09,AR,ATH,ELECTRICAL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,-1.74,0.42,20.23,20.74,41274.0,852.0,204022.0,446.0,33.49,55.0,M-MC,4.01,189.0,0.02,1.48,12.43,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,3.11,-2.64,113.55,107.91,163225.0,-3897.0,143747.0,1641.55,-20.96,58.0,M-SC,11.55,220.0,-0.02,1.04,8.15,OX40N,ATH,CABLES
66,SIS,-0.51,-21.47,54.41,21.26,47444.0,-23835.0,87197.0,528.00,2060.62,64.0,H-SC,4.57,163.0,-0.50,0.63,20.21,OX40N,NTT,MISC
72,TATAELXSI,-4.70,-25.75,77.81,32.03,76167.0,-33947.0,97888.0,9161.00,-19.73,39.0,H-SC,6.38,157.0,-0.45,0.71,9.64,OX40N,NTT,IT
77,TTKPRESTIG,1.63,-26.42,36.03,0.09,26717.0,-26624.0,74153.0,770.00,72.04,40.0,M-SC,8.21,253.0,-1.00,0.54,2.06,OX40N,NTT,DURABLES
48,KANSAINER,-1.91,-28.53,58.77,13.47,113270.0,-76932.0,192735.0,340.00,-70.65,33.0,H-SC,6.07,159.0,-0.68,1.39,0.00,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,-1.74,0.42,20.23,20.74,41274.0,852.0,204022.0,446.00,33.49,55.0,M-MC,4.01,189.0,0.02,1.48,12.43,XY25,NTT,FINANCE
18,COALINDIA,-0.49,5.68,12.82,19.23,20385.0,8548.0,159008.0,484.83,24.27,53.0,L-LC,12.42,182.0,0.42,1.15,28.16,XY25,ATH,MINING
1,ABB,0.45,6.34,36.89,45.57,102631.0,16589.0,278208.0,7934.00,-31.69,72.0,H-MC,4.47,121.0,0.16,2.01,23.54,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.45,6.34,36.89,45.57,102631.0,16589.0,278208.0,7934.00,-31.69,72.0,H-MC,4.47,121.0,0.16,2.01,23.54,AR,NTT,ELECTRICAL
26,FINCABLES,3.11,-2.64,113.55,107.91,163225.0,-3897.0,143747.0,1641.55,-20.96,58.0,M-SC,11.55,220.0,-0.02,1.04,8.15,OX40N,ATH,CABLES
39,INDIAMART,-2.38,-4.24,115.87,106.72,136854.0,-5226.0,118110.0,4810.62,-52.54,51.0,H-SC,8.35,138.0,-0.04,0.85,18.15,AR,ATH,MISC
84,WIPRO,-1.32,-5.54,82.37,72.27,131840.0,-9390.0,160058.0,420.00,-18.78,36.0,M-LC,3.15,101.0,-0.07,1.16,2.88,XR,NTT,IT
50,MASFIN,2.22,-1.87,23.88,21.57,22961.0,-1830.0,96150.0,397.04,-15.54,54.0,H-SC,3.25,168.0,-0.08,0.70,40.31,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.32,-45.24,146.91,35.20,103511.0,-58211.0,70459.0,2096.00,-7.58,24.0,X-SC,21.11,93.0,-0.56,0.51,0.13,X40,NTT,FOOTWEAR
6,ASIANPAINT,-1.36,-16.14,44.23,20.95,93379.0,-40647.0,211121.0,3460.25,-16.94,27.0,X-LC,9.15,36.0,-0.44,1.53,13.74,X40,ATH,PAINTS
81,VBL,0.08,-11.64,53.54,35.67,149426.0,-36749.0,279093.0,671.64,-22.18,27.0,X-LC,0.70,5.0,-0.25,2.02,0.61,X40N,ATH,FMCG
13,BERGEPAINT,-0.31,-16.12,44.42,21.14,84706.0,-36645.0,190694.0,680.00,-22.05,31.0,X-MC,8.74,75.0,-0.43,1.38,1.72,XY24,NTT,PAINTS
42,INFY,-1.22,2.22,31.33,34.24,100206.0,6953.0,319841.0,1972.00,-20.32,31.0,X-LC,6.07,6.0,0.07,2.31,10.69,X40,NTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,0.08,-11.64,53.54,35.67,149426.0,-36749.0,279093.0,671.64,-22.18,27.0,X-LC,0.70,5.0,-0.25,2.02,0.61,X40N,ATH,FMCG
73,TCS,-2.24,-19.59,47.43,18.55,144257.0,-74089.0,304148.0,4311.58,-30.74,34.0,X-LC,0.72,3.0,-0.51,2.20,3.47,X40,ATH,IT
5,ANGELONE,-0.67,7.59,15.44,24.20,50707.0,23162.0,328412.0,3033.00,53.38,55.0,X-SC,1.51,99.0,0.46,2.38,37.54,X40N,NTT,FINANCE
21,DABUR,0.41,-1.00,45.16,43.70,110904.0,-2488.0,245580.0,735.00,-9.24,47.0,X-MC,2.06,73.0,-0.02,1.78,13.90,XY24,BTT,FMCG
34,HINDUNILVR,2.17,-6.42,21.47,13.67,51646.0,-16513.0,240550.0,2922.00,-13.44,56.0,X-LC,2.42,9.0,-0.32,1.74,13.56,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.32,-45.24,146.91,35.20,103511.0,-58211.0,70459.0,2096.00,-7.58,24.0,X-SC,21.11,93.0,-0.56,0.51,0.13,X40,NTT,FOOTWEAR
81,VBL,0.08,-11.64,53.54,35.67,149426.0,-36749.0,279093.0,671.64,-22.18,27.0,X-LC,0.70,5.0,-0.25,2.02,0.61,X40N,ATH,FMCG
8,AWL,-2.05,-31.82,130.55,57.19,290292.0,-103766.0,222361.0,485.00,-67.71,35.0,X-MC,16.59,58.0,-0.36,1.61,1.34,XY24,NTT,FMCG
3,ACC,-0.86,-30.16,135.05,64.15,224426.0,-71771.0,166180.0,3906.00,-58.93,41.0,X-SC,7.84,84.0,-0.32,1.20,1.58,XY24,BTT,CEMENT
13,BERGEPAINT,-0.31,-16.12,44.42,21.14,84706.0,-36645.0,190694.0,680.00,-22.05,31.0,X-MC,8.74,75.0,-0.43,1.38,1.72,XY24,NTT,PAINTS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-2.35,-29.58,102.85,42.84,47080.0,-19231.0,45775.0,424.00,-53.72,55.0,X-SC,21.23,83.0,-0.41,0.33,10.11,XY24,NTT,MISC
58,RELAXO,-0.66,-52.05,217.67,52.31,151494.0,-75562.0,69598.0,1176.00,-48.99,41.0,X-SC,12.48,92.0,-0.50,0.50,3.49,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.32,-45.24,146.91,35.20,103511.0,-58211.0,70459.0,2096.00,-7.58,24.0,X-SC,21.11,93.0,-0.56,0.51,0.13,X40,NTT,FOOTWEAR
51,MEDANTA,2.97,-4.76,30.28,24.08,36610.0,-6046.0,120904.0,1486.00,-12.55,53.0,X-SC,11.28,91.0,-0.17,0.87,10.74,XY24,NTT,HEALTHCARE
35,HONAUT,-1.62,-21.47,82.51,43.32,105530.0,-34972.0,127900.0,58357.33,-32.50,41.0,X-SC,9.44,90.0,-0.33,0.93,3.85,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,4.22,-20.76,39.81,10.78,75808.0,-49894.0,190424.0,452.00,-52.60,43.0,X-LC,18.74,1.0,-0.66,1.38,4.89,X40,NTT,FMCG
75,TMPV,-1.38,-24.37,62.60,22.98,106950.0,-55051.0,170847.0,600.00,-21.86,59.0,X-LC,4.25,2.0,-0.51,1.24,9.22,XY24,NTT,AUTO
73,TCS,-2.24,-19.59,47.43,18.55,144257.0,-74089.0,304148.0,4311.58,-30.74,34.0,X-LC,0.72,3.0,-0.51,2.20,3.47,X40,ATH,IT
81,VBL,0.08,-11.64,53.54,35.67,149426.0,-36749.0,279093.0,671.64,-22.18,27.0,X-LC,0.70,5.0,-0.25,2.02,0.61,X40N,ATH,FMCG
42,INFY,-1.22,2.22,31.33,34.24,100206.0,6953.0,319841.0,1972.00,-20.32,31.0,X-LC,6.07,6.0,0.07,2.31,10.69,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.75,-32.49,96.85,32.89,50479.0,-25086.0,52121.0,1800.00,-743.79,56.0,L-MC,11.22,259.0,-0.50,0.38,43.54,XR,NTT,BANKS
67,SONACOMS,-2.54,-11.61,57.36,39.09,51289.0,-11743.0,89416.0,804.02,-30.90,61.0,M-SC,1.85,227.0,-0.23,0.65,26.49,AR,ATH,AUTO
50,MASFIN,2.22,-1.87,23.88,21.57,22961.0,-1830.0,96150.0,397.04,-15.54,54.0,H-SC,3.25,168.0,-0.08,0.70,40.31,XR,ATH,FINANCE
14,BSOFT,-5.09,-20.76,90.28,50.79,100626.0,-29193.0,111460.0,831.70,1.66,54.0,H-SC,6.84,171.0,-0.29,0.81,29.78,XR,ATH,IT
18,COALINDIA,-0.49,5.68,12.82,19.23,20385.0,8548.0,159008.0,484.83,24.27,53.0,L-LC,12.42,182.0,0.42,1.15,28.16,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.45,6.34,36.89,45.57,102631.0,16589.0,278208.0,7934.00,-31.69,72.0,H-MC,4.47,121.0,0.16,2.01,23.54,AR,NTT,ELECTRICAL
27,GILLETTE,1.48,3.39,25.47,29.72,66863.0,8619.0,262515.0,10978.86,-14.94,67.0,X-SC,11.23,87.0,0.13,1.90,21.48,X40,ATH,FMCG
5,ANGELONE,-0.67,7.59,15.44,24.20,50707.0,23162.0,328412.0,3033.00,53.38,55.0,X-SC,1.51,99.0,0.46,2.38,37.54,X40N,NTT,FINANCE
67,SONACOMS,-2.54,-11.61,57.36,39.09,51289.0,-11743.0,89416.0,804.02,-30.90,61.0,M-SC,1.85,227.0,-0.23,0.65,26.49,AR,ATH,AUTO
11,BANDHANBNK,0.42,-23.77,152.41,92.42,323643.0,-66209.0,212350.0,400.00,49.43,64.0,H-SC,9.16,169.0,-0.20,1.54,22.72,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.30
1,25,44.65
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.42
MC    30.90
LC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.58
X40      23.49
X40N     15.04
AR        8.98
XR        8.85
XY25      8.79
OX40N     7.42
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.16
H-SC    22.92
X-LC    20.26
M-SC    11.41
X-SC    10.34
H-MC     4.88
M-MC     1.48
H-LC     1.16
M-LC     1.16
L-LC     1.15
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.01,-14.11,52.46
FINANCE,13.51,-12.02,56.10
IT,12.25,-26.28,89.37
MISC,7.13,-31.91,85.87
ELECTRICAL,6.15,-10.46,50.76
PAINTS,5.20,-28.42,47.50
INSURANCE,4.75,-3.25,38.58
PHARMA,4.02,-5.68,38.95
AUTO,2.98,-27.68,71.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3298114.0,21
AR,1330001.0,10
XR,1272630.0,12
X40,1250627.0,15
X40N,1042564.0,10
OX40N,767728.0,10
XY25,405431.0,6
SR,296589.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3630021.0,24
X-MC,1686578.0,16
M-SC,1484256.0,15
X-LC,1091907.0,12
X-SC,862561.0,9
H-MC,403576.0,3
L-SC,187477.0,2
M-LC,131840.0,1
H-LC,73330.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1245374.0      6
           AR         941157.0      5
M-SC       XY24       835817.0      6
H-SC       XR         813657.0      7
X-MC       X40        556091.0      7
X-LC       X40        524162.0      6
X-MC       XY24       485902.0      3
           X40N       418240.0      4
X-SC       X40N       384071.0      4
M-SC       OX40N      342664.0      5
H-SC       OX40N      333244.0      4
X-SC       XY24       308116.0      3
H-SC       SR         296589.0      2
X-LC       X40N       240253.0      2
X-MC       XY25       226345.0      2
H-MC       XY24       212840.0      1
X-LC       XY24       210065.0      2
H-MC       AR         190736.0      2
M-SC       XR         180997.0      2
X-SC       X40        170374.0      2
M-LC       XR         131840.0      1
M-SC       AR         124778.0      2
X-LC       XY25       117427.0      2
L-SC       XR          95657.0      1
           OX40N       91820.0      1
H-LC       AR          73330.0      1
L-MC       XR          50479.0      1
M-MC       XY25        41274.0      1
L-LC       XY25        20385.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
